## Download dataset and model

## Google Colab import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download data
!gdown --id 1RhJmQ2eBo7KYoXpCQ3mn2A02YUrFo4ID
# Download torchtrainer library
!gdown --id 1DzSFLvGsyxmx2j1N-l5k0P_4Bx2i5g7D
# Download learner vessel
!gdown --id 1EP93TNX180IGYefQnVaeL4YeXVtZYjjb

In [ ]:
# Copy torchtrainer library
!cp '/content/drive/MyDrive/Iniciação Científica/Wesley Galvão.lnk/Notebooks/torchtrainer' -r '/content/'
# Copy model checkpoint 
!cp '/content/drive/MyDrive/Iniciação Científica/Wesley Galvão.lnk/Modelos/learner_vessel.tar' '/content/'
# Copy image dataset
!cp '/content/drive/MyDrive/Iniciação Científica/Wesley Galvão.lnk/Notebooks/Dados/data' -r '/content'

## Import libraries

In [1]:
# Progress bar
! pip install pyprog
# Install imgaug
!pip install imgaug==0.4

     |████████████████████████████████| 948 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 11.2 MB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 60.5 MB 29.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 21.3 MB/s eta 0:00:01
     |████████████████████████████████| 42.1 MB 28.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 29.8 MB/s eta 0:00:01
     |████████████████████████████████| 14.0 MB 16.8 MB/s eta 0:00:01
     |████████████████████████████████| 4.3 MB 39.0 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 29.2 MB/s eta 0:00:01
     |████████████████████████████████| 191 kB 19.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 36.6 MB/s eta 0:00:01
     |████████████████████████████████| 930 kB 31.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 35.5 MB/s eta 0:00:01


In [28]:
import pyprog

# Initial imports and device setting
from pathlib import Path
from functools import partial


import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import torch
import torch.nn as nn
from torch.nn.functional import interpolate 
from torch.functional import F
import imgaug.augmenters as iaa

# Libraries for graph
import networkx.drawing as draw
import networkx as nx

import math 
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px

from plotly.subplots import make_subplots
import plotly.graph_objects as go


from torchtrainer.imagedataset import ImageSegmentationDataset
from torchtrainer import img_util
from torchtrainer import transforms
from torchtrainer.models.resunet import ResUNet
from torchtrainer.module_util import ActivationSampler

# Resunet Explorator library
from resunetexplorer.layer_extractor import ExtractResUNetLayers
from resunetexplorer.maps_extractor import ExtractResUNetMaps


import scipy
from scipy import ndimage, misc

# PCA module
from sklearn.decomposition import PCA


use_cuda = False

if use_cuda and torch.cuda.is_available():
    device = torch.device('cuda')
    dev_info = torch.cuda.get_device_properties(device)
    print(dev_info)
else:
    device = torch.device('cpu')

# Functions

In [2]:
def dataset_creation(root_dir_, img_dir_, label_dir_):
  """
  Load the dataset given the complet path.
  """
  # Dataset creation
  def img_name_to_label(filename):
      return filename.split('.')[0] + '.png'

  root_dir = Path(root_dir_)
  img_dir = root_dir/img_dir_
  label_dir = root_dir/label_dir_

  # Data transformations
  imgaug_seq = iaa.Sequential([
      iaa.CLAHE(clip_limit=6, tile_grid_size_px=12)
  ])    
  imgaug_seq = transforms.translate_imagaug_seq(imgaug_seq)
  valid_transforms = [transforms.TransfToImgaug(), imgaug_seq, transforms.TransfToTensor(), 
                      transforms.TransfWhitten(67.576, 37.556)]

  img_opener = partial(img_util.pil_img_opener, channel=None)
  label_opener = partial(img_util.pil_img_opener, is_label=True)
  dataset = ImageSegmentationDataset(img_dir, label_dir, name_to_label_map=img_name_to_label, img_opener=img_opener, 
                                    label_opener=label_opener, transforms=valid_transforms)

  return dataset

In [3]:
def load_model_checkpoint(path, device):
  """
  Load the model from a checkpoint given a path to file and the device which 
  will process the model
  """
  checkpoint = torch.load(path, map_location=torch.device(device))
  model = ResUNet(num_channels=1, num_classes=2) 
  model.load_state_dict(checkpoint['model_state'])
  model.eval()
  model.to(device);

  return model

#### Big Matrix

In [4]:
def big_matrix(nmaps, layer, img_idx):
  """ """
  # Sample the activations on one of the layers
  sampler = ActivationSampler(layer)

  # Apply network to image with index = img_idx
  img, _ = dataset[img_idx]

  with torch.no_grad():
    img = img.to(device)[None]
    model(img);

  # Get activation maps images for specified layer
  layer_activation = sampler().detach().to('cpu')[0]

  # Get flatten image size
  tensor_size = len(torch.flatten(layer_activation[0]))
  # Create an empty tensor matrix 
  big_matrix = torch.empty(size=(nmaps, tensor_size))
  # Reshape the layer_activation to build the big_matrix of size nmaps X tensor_size
  big_matrix = layer_activation.reshape(nmaps, -1)

  """
  Alternative using for loop

  # Stack all flattened images to build a matrix of size nmaps X tensor_size
  for i in range(nmaps):
    big_matrix[i] = torch.flatten(layer_activation[i])
  """
  
  return big_matrix 

In [5]:
def big_matrix_masked(nmaps, layer, img_idx):
  """ Big matrix for masked feature maps"""
  # Sample the activations on one of the layers
  sampler = ActivationSampler(layer)

  # Apply network to image with index = img_idx
  img, _ = dataset[img_idx]

  with torch.no_grad():
    img = img.to(device)[None]
    model(img);

  # Get activation maps images for specified layer
  layer_activation = sampler().detach().to('cpu')[0]

  # apply dilation on label with iteration level = 7
  dilated_label = scipy.ndimage.binary_dilation(label, iterations = 7)

  # Mask all feature maps of layer_activation
  layer_activation_masked = layer_activation*dilated_label

  # Get flatten image size
  tensor_size = len(torch.flatten(layer_activation_masked[0]))
  # Create an empty tensor matrix 
  big_matrix = torch.empty(size=(nmaps, tensor_size))
  # Reshape the layer_activation to build the big_matrix of size nmaps X tensor_size
  big_matrix = layer_activation_masked.reshape(nmaps, -1)

  """
  Alternative using for loop

  # Stack all flattened images to build a matrix of size nmaps X tensor_size
  for i in range(nmaps):
    big_matrix[i] = torch.flatten(layer_activation[i])
  """
  
  return big_matrix, layer_activation_masked

In [6]:
def big_matrix_stats(dataset, layer, nmaps, path):

  """ """

  # Sample the activations on one of the layers
  sampler = ActivationSampler(layer)

  # Apply network to image with index = 0
  img, _ = dataset[0]

  with torch.no_grad():
    img = img.to(device)[None]
    model(img);

  # Get activation maps images for specified layer
  layer_activation = sampler().detach().to('cpu')[0]

  # Get dataset len
  dataset_len = len(dataset)
  #dataset_len = 4

  #----------- Average ------------------
  # Create a matrix numpy array. Size nmaps X nmaps
  corr_matrix = np.empty(shape=[nmaps, nmaps])
  # Create an empty numpy array
  corr_sum = np.zeros([nmaps, nmaps])
  # Get flatten image size
  tensor_size = len(torch.flatten(layer_activation[0]))
  # Create two empty tensors 
  matrix = torch.zeros([nmaps, tensor_size])
  matrix_sum = torch.zeros([nmaps, tensor_size])

  # Create Object to progress bar
  prog = pyprog.ProgressBar(" ", "", dataset_len)
  # Print Task name
  print('Computing average of Big Matrix and Correlation Matrix: \n')
  # Update Progress Bar
  prog.update()

  for idx in range(dataset_len):
    # Get the big_matrix
    matrix = big_matrix(nmaps, layer, idx)
    # Sum of all matrices
    matrix_sum = matrix_sum + matrix
    # Pearson Correlation
    corr_matrix = np.abs(np.corrcoef(matrix))
    # Sum of all correlation matrices
    corr_sum =  corr_sum + corr_matrix
    # Get filename of current image  
    filename = dataset.img_file_paths[idx].stem
    # Save current Correlation Matrix as a figure
    #plot_corr_matrix(path, filename, corr_matrix)

    # Set current status
    prog.set_stat(idx + 1)
    # Update Progress bar again
    prog.update()  
  
  # Make the Progress Bar final
  prog.end()

  # Compute big matrix average  
  matrix_avg = matrix_sum/dataset_len
  # Compute big matrix correlation average
  corr_avg = corr_sum/dataset_len

  #----------- Standard deviation------------------

  # Create two empty tensors 
  matrix = torch.zeros([nmaps, tensor_size])
  matrix_std = torch.zeros([nmaps, tensor_size])

  # Create an empty numpy array
  corr_std = np.zeros([nmaps, nmaps])
  corr = np.zeros([nmaps, nmaps])

  # Create Object to progress bar
  prog = pyprog.ProgressBar(" ", "", dataset_len)
  # Print Task name
  print('Computing standard deviation of Big Matrix and Correlation Matrix: \n')
  # Update Progress Bar
  prog.update()


  for idx in range(dataset_len):
    # Get the big_matrix
    matrix = big_matrix(nmaps, layer, idx)
    # Compute the sum part of standard deviation of big matrix
    matrix_std = matrix_std + torch.float_power((matrix - matrix_avg),2)
    # Pearson Correlation of big_matrix
    corr_matrix = np.abs(np.corrcoef(matrix))
    # Compute the sum part of standard deviation of correlation matrix
    corr_std = corr_std + np.float_power((corr_matrix - corr_avg),2)
    # Set current status
    prog.set_stat(idx + 1)
    # Update Progress bar again
    prog.update()
    
    #print('Desvio padrão: \n')
    #print(idx, matrix_sum.max(), corr_sum.max())

  # Make the Progress Bar final
  prog.end()

  # Compute standard deviation for both big matrix and correlation map
  corr_std = np.sqrt(corr_std/(dataset_len-1))
  matrix_std = torch.sqrt(matrix_std/(dataset_len-1))

  return matrix_avg, matrix_std, corr_avg, corr_std


#### PCA

In [7]:
def pca(n, matrix):
  # Set 2 principals components
  pca = PCA(n_components=2, whiten=True)

  # Transform big_matrix to PCA
  principalComponents = pca.fit_transform(matrix)

  # Transform to dataframe
  pca_df = pd.DataFrame(data = principalComponents
              , columns = ['principal_component_1', 'principal_component_2'])
  
  print(pca_df.head())

  # Plot PCA scatter plot
  fig = plt.figure(figsize = (8,8))
  fig.suptitle('Scartter plot of PCA')
  ax = sns.scatterplot(data = pca_df, x = 'principal_component_1', 
                y ='principal_component_2' )
  return pca_df

#### Plot Correlation Matrix

In [8]:
def plot_corr_matrix(path, filename, corr_matrix):
  """ """
  # Convert to Dataframe
  corr_df = pd.DataFrame(corr_matrix)
  # Set figure size
  fig = plt.figure(figsize=[150, 150])
  # Set figure title
  plt.title(filename+'-Activation_Maps_Correlation_encoder.resblock1',  fontsize=30)
  # Plot a matrix heatmap
  sns.heatmap(corr_df, cmap='coolwarm', annot = False)
  sns.set(font_scale = 2)
  # save the figure
  plt.savefig(path+filename+'-Activation_Maps_Correlation_encoder.resblock_mid.jpg', dpi=100, bbox_inches='tight')
  # Close figure
  plt.close(fig)


#### Feature Maps Interpolation

In [9]:
def feature_maps_interp(layer, mode_ = 'linear', scale_factor_ = 2):
  """Feature maps interpolation"""

  if mode_ == 'linear':
    # Run interpolation on feature maps with chosen scale factor
    feature_maps_interpolated = F.interpolate(layer, 
                                              scale_factor=scale_factor_, 
                                              mode = mode_)
    
    feature_maps_interpolated = feature_maps_interpolated.permute(0, 2, 1)

    feature_maps_interpolated = F.interpolate(feature_maps_interpolated, 
                                              scale_factor=scale_factor_, 
                                              mode = mode_)
    
    feature_maps_interpolated = feature_maps_interpolated.permute(0, 2 , 1)
  
    
  if mode_ == 'bicubic':
    # Change tensor dimension to [batch_size == 1, channels, height, width]
    layer = layer[None]
    # Run interpolation on feature maps with chosen scale factor
    feature_maps_interpolated = F.interpolate(layer, 
                                              scale_factor=scale_factor_, 
                                              mode = mode_)
    feature_maps_interpolated = torch.squeeze(feature_maps_interpolated)

  return feature_maps_interpolated

In [10]:
def feature_maps_interp2(layer, img_idx, scale_factor_):
  """"""

  # Sample the activations on one of the layers
  sampler = ActivationSampler(layer)
  # Apply network to image with index = img_idx
  img, _ = dataset[img_idx]

  with torch.no_grad():
    img = img.to(device)[None]
    model(img);

  # Get activation maps images for specified layer
  feature_maps_interpolated = sampler().detach().to('cpu')[0]

  # Run interpolation on feature maps with chosen scale factor
  feature_maps_interpolated = F.interpolate(feature_maps_interpolated, 
                                            scale_factor=scale_factor_, 
                                            mode = 'linear')
  
  feature_maps_interpolated = feature_maps_interpolated.permute(0, 2, 1)

  feature_maps_interpolated = F.interpolate(feature_maps_interpolated, 
                                            scale_factor=scale_factor_, 
                                            mode = 'linear')
  
  feature_maps_interpolated = feature_maps_interpolated.permute(0, 2, 1)

  return feature_maps_interpolated

In [11]:
def feature_maps_interp3(layer, img_idx, zoom_):
  # Sample the activations on one of the layers
  sampler = ActivationSampler(layer1)
  # Apply network to image with index = img_idx
  img, _ = dataset[3]

  with torch.no_grad():
    img = img.to(device)[None]
    model(img);

  # Get activation maps images for specified layer
  feature_maps = sampler().detach().to('cpu')[0]
  feature_maps_interpolated = ndimage.zoom(feature_maps, zoom_)

  return feature_maps_interpolated

In [12]:
def feature_maps_sampling(layer, img_idx, kernel_size_):

  # Sample the activations on one of the layers
  sampler = ActivationSampler(layer1)
  # Apply network to image with index = img_idx
  img, _ = dataset[3]

  with torch.no_grad():
    img = img.to(device)[None]
    model(img);

  # Get activation maps images for specified layer
  feature_maps = sampler().detach().to('cpu')[0]
  # Change tensor dimension to [batch_size == 1, channels, height, width]
  feature_maps_sampled = feature_maps[None]
  # Run interpolation on feature maps with chosen scale factor
  feature_maps_sampled = torch.nn.functional.avg_pool2d(feature_maps_sampled, 
                                            kernel_size = kernel_size_)

  return feature_maps_sampled

In [13]:
def feature_maps_sampling2(layer, img_idx, kernel_size_):

  # Sample the activations on one of the layers
  sampler = ActivationSampler(layer1)
  # Apply network to image with index = img_idx
  img, _ = dataset[3]

  with torch.no_grad():
    img = img.to(device)[None]
    model(img);

  # Get activation maps images for specified layer
  feature_maps = sampler().detach().to('cpu')[0]
  # Change tensor dimension to [batch_size == 1, channels, height, width]
  feature_maps_sampled = feature_maps[None]
  # Run interpolation on feature maps with chosen scale factor
  feature_maps_sampled = torch.nn.functional.max_pool2d(feature_maps_sampled, 
                                            kernel_size = kernel_size_)

  return feature_maps_sampled

#### Graph

In [14]:
def create_graph(corr_matrix, threshold):
    """Create graph from a correlation matrix"""
    
    # Keep correlations larger than threshold
    adjacency_mat = np.abs(corr_matrix)>=threshold
    
    adjacency_mat[np.diag_indices_from(adjacency_mat)] = 0
    edges_indices = np.nonzero(adjacency_mat)
    graph = nx.Graph(adjacency_mat*corr_matrix)
    
    correlations = corr_matrix[edges_indices]
    edges = list(zip(*edges_indices))
    weight = dict(zip(edges, correlations))
    nx.set_edge_attributes(graph, weight, 'weight')
    
    return graph

def show_graph(graph, ax=None):
    """Show graph"""
    
    weights = nx.get_edge_attributes(graph, 'weight')
    edge_labels = {k:f'{v:.2f}' for k, v in weights.items()}

    pos = draw.fruchterman_reingold_layout(graph, weight=None)
    nodes_names = {idx:idx for idx in range(0, len(graph))}

    if ax is None:
        fig = plt.figure(figsize=[25, 25])
        ax = fig.add_subplot(111)
    draw.draw_networkx_edges(graph, pos, width=1.0, edge_color='k', ax=ax)
    draw.draw_networkx_nodes(graph, pos, node_size=200, node_color='C0', ax=ax)
    draw.draw_networkx_labels(graph, pos, labels=nodes_names, font_size=8, ax=ax)
    draw.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels, label_pos=0.5, font_size=8, rotate=True, ax=ax)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

Image normalization

## Show activation maps

In [15]:
def show_activation_maps(dataset, layer, out_channels, img_idx):
  # Sample the activations on one of the layers
  sampler = ActivationSampler(layer)

  img, _ = dataset[img_idx]
  filename = dataset.img_file_paths[img_idx].stem

  with torch.no_grad():
      img = img.to(device)[None]
      model(img);
      
  layer_activation = sampler().to('cpu')[0]
  plt.figure(figsize=[50, 50])

  nrows = out_channels/4
  ncols = 16

  with PdfPages(r'Feature_maps_resblock1_'+filename+'.pdf') as export_pdf:
    for idx in range(out_channels):
        fig = plt.subplot(8, 8, idx+1)
        # displaying the title
        plt.title(idx, fontsize=20)
        ax = plt.imshow(layer_activation[idx], 'gray')
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)

        plt.subplots_adjust(wspace=0.03, hspace=0.03)
    plt.tight_layout()

    export_pdf.savefig(dpi=250, orientation='portrait')

In [16]:
def show_correlated_maps_pairs(df_corr, col_1, col_2, fm_layer1, fm_layer2):
  """
  Function to print a set of correlated feature maps from different layers. 
  The print is pair by pair with the respective correlation level. 

  - df_corr : dataframe that contains a set of indexes of correlated feature maps from different layers and the correlation level. 
  - col_1 : Name of the first column of df_corr
  - col_2 : Name of the second column of df_corr
  - Name of the first column of df_corr
  - fm_layer1: A tensor of feature maps. Must be the same set of feature maps realated to col_1
  - fm_layer2: A tensor of feature maps. Must be the same set of feature maps realated to col_2
  """
  nrows = df_corr.shape[0]

  for row in range(nrows):  
    fig = plt.figure(figsize=[11, 13])

    idx = 1
    fig = plt.subplot(2, 2, idx)
    # Get correlation value
    corr_value = round(df_corr['correlation'].iloc[row], 2)
    # Get layer_activation
    idx1 = df_corr[col_1].iloc[row]
    # displaying the title
    plt.title(f'Masked feature map {col_1} - {str(idx1)} corr: {str(corr_value)}', fontsize = 8)
    # Plot figure
    ax = plt.imshow(layer1_masked[idx1], 'gray')
    # Hide axis
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    
    plt.subplots_adjust(wspace=0.03, hspace=0)
    plt.tight_layout()

    fig = plt.subplot(2, 2, idx+1)
    # Get layer_activation
    idx2 = df_corr[col_2].iloc[row]
    # displaying the title
    plt.title(f'Masked feature map {col_2} - {str(idx2)} corr: {str(corr_value)}', fontsize = 8)
    # Plot figure    
    ax = plt.imshow(layer2_masked[idx2], 'gray')
    # Hide axis
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)

    plt.subplots_adjust(wspace=0.03, hspace=0)
    plt.tight_layout()

## Similarity_evaluation

In [17]:
def similarity_evaluation(feature_map1, feature_map2, feature_map1_name, feature_map2_name, df_results):
  # Euclidian distance
  ed_fm1_fm2 = torch.sqrt(torch.sum((feature_map1-feature_map2)**2))

  # Flattening  
  feature_map1_1d = feature_map1.flatten()
  feature_map2_1d = feature_map2.flatten()

  # Cossine Distance
  cos = nn.CosineSimilarity(dim = 0, eps=1e-6)
  cos_fm1_fm2 = cos(feature_map1_1d, feature_map2_1d)

  results_dict = {
      "feature_map_1_id" : feature_map1_name, 
      "feature_map_2_id" : feature_map2_name,
      "euclidian distance" : ed_fm1_fm2.numpy(), 
      "cossine distance" : cos_fm1_fm2.numpy()
  }

  df_result = pd.DataFrame([results_dict])
  df_results = pd.concat([df_results, df_result])

  return df_result , df_results


In [18]:
# based on:  https://www.geeksforgeeks.org/how-to-normalize-images-in-pytorch/

def image_normalize(img_tensor):
  mean, std = img_tensor.mean(), img_tensor.std()

  transform_norm = transforms.Compose([
      transforms.ToTensor(), 
      transforms.Normalize(mean, std)                            
  ])
  # Convert tensor to np
  img_np = np.array(img_tensor)
  # Get normalized image
  img_normalized = transform_norm(img_np)
  # Remove batch dimension
  img_normalized_squeezed = torch.squeeze(img_normalized)
  # Return normalized image
  return img_normalized_squeezed

In [19]:
def similarity(feature_map1, feature_map2):
  # Euclidian distance
  ed_fm1_fm2 = torch.sqrt(torch.sum((feature_map1-feature_map2)**2))

  # Flattening  
  feature_map1_1d = feature_map1.flatten()
  feature_map2_1d = feature_map2.flatten()

  # Cossine Distance
  cos = nn.CosineSimilarity(dim = 0, eps=1e-6)
  cos_fm1_fm2 = cos(feature_map1_1d, feature_map2_1d)
  
  return ed_fm1_fm2, cos_fm1_fm2

In [20]:
def similarity_metrics_normalized(corr_matrix_reduced, layer):

  # Create the dataframe to store euclidian and cossine distance 
  similarity_metrics = pd.DataFrame(columns=("feature_map_1_id", "feature_map_2_id", "euclidian_distance", "cossine_distance"))
  # Get number of rows of correlation matrix
  nrows = corr_matrix_reduced.shape[0]

  # Iterate over the correlation matrix to get the correlated pairs and calculate both euclidian and cossine distance
  for i in range(nrows):

    # Get feature maps IDs of correlated pairs
    feature_map_1_id = corr_matrix_reduced['feature_map_1_id'].loc[i]
    feature_map_2_id = corr_matrix_reduced['feature_map_2_id'].loc[i]

    # Normalize the feature maps
    normalized_fm_1 = image_normalize(layer[feature_map_1_id])
    normalized_fm_2 = image_normalize(layer[feature_map_2_id])

    # Compute Euclidian distance
    ed_fm1_fm2 = torch.sqrt(torch.sum((normalized_fm_1-normalized_fm_2)**2))

    # Flattening  
    normalized_fm_1 = normalized_fm_1.flatten()
    normalized_fm_2 = normalized_fm_2.flatten()

    # Compute Cossine Distance
    cos = nn.CosineSimilarity(dim = 0, eps=1e-6)
    cos_fm1_fm2 = cos(normalized_fm_1, normalized_fm_2)
    
    # A dictionary to store the data
    results_dict = {
        "feature_map_1_id" : feature_map_1_id, 
        "feature_map_2_id" : feature_map_2_id,
        "euclidian_distance" : ed_fm1_fm2.numpy(), 
        "cossine_distance" : cos_fm1_fm2.numpy()
    }

    # Convert the dict to dataframe
    df_result = pd.DataFrame([results_dict])
    # Concact df_result with similarity_metrics to append new row
    similarity_metrics = pd.concat([similarity_metrics, df_result])

  # Reset indexes
  similarity_metrics.reset_index(drop = True, inplace = True)

  return similarity_metrics


In [21]:
def similarity_metrics_unormalized(corr_matrix_reduced, layer):

  # Create the dataframe to store euclidian and cossine distance 
  similarity_metrics = pd.DataFrame(columns=("feature_map_1_id", "feature_map_2_id", "euclidian_distance", "cossine_distance"))
  # Get number of rows of correlation matrix
  nrows = corr_matrix_reduced.shape[0]

  # Iterate over the correlation matrix to get the correlated pairs and calculate both euclidian and cossine distance
  for i in range(nrows):

    # Get feature maps IDs of correlated pairs
    feature_map_1_id = corr_matrix_reduced['feature_map_1_id'].loc[i]
    feature_map_2_id = corr_matrix_reduced['feature_map_2_id'].loc[i]

    # Normalize the feature maps
    feature_map_1 = layer[feature_map_1_id]
    feature_map_2 = layer[feature_map_2_id]

    # Compute Euclidian distance
    ed_fm1_fm2 = torch.sqrt(torch.sum((feature_map_1-feature_map_2)**2))

    # Flattening  
    feature_map_1 = feature_map_1.flatten()
    feature_map_2 = feature_map_2.flatten()

    # Compute Cossine Distance
    cos = nn.CosineSimilarity(dim = 0, eps=1e-6)
    cos_fm1_fm2 = cos(feature_map_1, feature_map_2)
    
    # A dictionary to store the data
    results_dict = {
        "feature_map_1_id" : feature_map_1_id, 
        "feature_map_2_id" : feature_map_2_id,
        "euclidian_distance" : ed_fm1_fm2.numpy(), 
        "cossine_distance" : cos_fm1_fm2.numpy()
    }

    # Convert the dict to dataframe
    df_result = pd.DataFrame([results_dict])
    # Concact df_result with similarity_metrics to append new row
    similarity_metrics = pd.concat([similarity_metrics, df_result])

  # Reset indexes
  similarity_metrics.reset_index(drop = True, inplace = True)

  return similarity_metrics


## Sampler to CPU

In [22]:
def get_feature_maps(img_idx, layer):
  """
  Function that receives an image index and a ResUNet layer, send the 
  feature maps of its respective image and layer to CPU and returns the 
  feature maps.  
  """
  sampler = ActivationSampler(layer)

  img, label = dataset[img_idx]
  with torch.no_grad():
      img = img.to(device)[None]
      model(img);
      
  layer_feature_maps = sampler().to('cpu')[0]
  
  return layer_feature_maps



## Feature maps masking

In [23]:
def feature_maps_masking(layer_feature_maps, label, iterations_level):
  """
  Apply the dilation and masking over all the feature maps of ResUNet layer. 
  The dilation uses iterations_level to define the level of iterations.  
  """
  # apply dilation on label with iterations_level
  dilated_label = scipy.ndimage.binary_dilation(label, iterations = iterations_level)

  # Mask all feature maps of layer_feature_maps
  feature_maps_masked = layer_feature_maps*dilated_label

  return feature_maps_masked

## Feature maps correlation

In [24]:
def feature_maps_correlation(layer_1_fm, layer_2_fm, layer_1_name, layer_2_name, n_maps1, n_maps2):

  
  fm_correlation = pd.DataFrame(columns=(layer_1_name+'_fm_id', layer_2_name+'_fm_id', 'correlation'))

  # Create Object to progress bar
  prog = pyprog.ProgressBar(" ", "", n_maps1)
  # Print Task name
  print('Computing feature maps correlation: \n')
  # Update Progress Bar
  prog.update()


  for map_idx1 in range(n_maps1):
    layer_1_map_1d = layer_1_fm[map_idx1].flatten()

    for map_idx2 in range(n_maps2):   
      
      layer_2_map_1d = layer_2_fm[map_idx2].flatten()
      corr = np.corrcoef(layer_1_map_1d, layer_2_map_1d)[0][1]

      fm_correlation_dict = {
          layer_1_name+'_fm_id' : map_idx1, 
          layer_2_name+'_fm_id' : map_idx2,
          'correlation'         : corr

      }

      # Convert the dict to dataframe
      df_result = pd.DataFrame([fm_correlation_dict])
      # Concact df_result with similarity_metrics to append new row
      fm_correlation = pd.concat([fm_correlation, df_result])
      
    # Set current status
    prog.set_stat(map_idx1 + 1)
    # Update Progress Bar
    prog.update()

  # Make the Progress Bar final
  prog.end()

  # Reset indexes
  fm_correlation.reset_index(drop = True, inplace = True)

  return fm_correlation


# Prototype


In [25]:
# Load dataset
dataset = dataset_creation('data', 'CD31(vessels)', 'labels')
# Model path
model_path = 'learner_vessel.tar'
# Load model
model = load_model_checkpoint(model_path, 'cuda')

AttributeError: 'NoneType' object has no attribute 'lower'

## ExtractResUNetLayers class

In [39]:
class ExtractResUNetLayers:
  """Layers extractor class for PyTorch ResUNet. 

    Receives the model, the network part (encoder or decoder) and the names of the layers to be extracted

    Parameters
    ----------
    model : torchtrainer.models.resunet.ResUNet
        Directory containing the images to be read
    network_part : string
        A string the represents the name of ResUNet Network part. network_part = 'encoder' or 'decoder'
    layer_names : list 
        Contains the names of layers belonging network_part. 
        e.g. If network_part == 'encoder', a possible list of layers is: ['resblock1', 'resblock2', ...]
    
  """

  def __init__(self, model:str, network_part:str, layer_names:list):
    self.model = model
    self.network_part = network_part
    self.layer_names = layer_names

    # Get model network part (encoder or decoder) 
    self.model_network_part = getattr(self.model, self.network_part)
  
  def get_number_maps(self, layer):
    """
    """   
    # Get number of feature maps of layer
    n_maps = layer.conv1.out_channels

    return n_maps
  
  def get_layers(self):
    """
    """
    layers_dict = {
      "layer_name":[], 
      "network_part":[],
      "n_maps":[], 
      "layer":[], 

    }

    for i, name in enumerate(self.layer_names):
      # Get desired layer from model 
      layer = getattr(self.model_network_part, name)
      n_maps = self.get_number_maps(layer)
      
      # Dict appending
      layers_dict["layer_name"].append(name)
      layers_dict["network_part"].append(self.network_part)      
      layers_dict["n_maps"].append(n_maps)
      layers_dict["layer"].append(layer)

    return layers_dict


In [40]:
# ExtractResUNetLayers test
layer_name = ['resblock1', 'resblock2', 'resblock3']
erl = ExtractResUNetLayers(model, 'encoder', layer_name)
layers = erl.get_layers()


AttributeError: 'dict' object has no attribute 'encoder'

## ExtractResUNetMaps Class

In [29]:
layers_name = ['resblock1', 'resblock2']
network_part = 'encoder'
img_idx = 3

erl = ExtractResUNetLayers(model, network_part, layer_name)
layers = erl.get_layers()


NameError: name 'model' is not defined

In [44]:

layers_fm_list = get_multiple_feature_maps(img_idx, layers['layer'])

maps_idx = [2,6,7,52]

show_feature_maps(img_idx, layers_name, layers_fm_list, maps_idx)

AttributeError: 'dict' object has no attribute 'encoder'

In [88]:
import timeit
runtimes = []
threads = [1] + [t for t in range(2, 49, 2)]
for t in threads:
    torch.set_num_threads(t)
    r = timeit.timeit(setup = "import torch; x = torch.randn(1024, 1024); y = torch.randn(1024, 1024)", stmt="torch.mm(x, y)", number=100)
    runtimes.append(r)
# ... plotting (threads, runtimes) ...